In [1]:
#importing needed stuff

import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.sparse import csr_matrix, find

#import the data
rating = pd.read_csv('testdata.csv')

#print the rating matrix
rating


,We,User2,User3,User4,User5,User6,User7,User8
Movie1,0,1,0,0,0,1,1,0
Movie2,0,1,0,0,0,1,1,0
Movie3,1,1,1,0,0,1,1,0
Movie4,1,1,0,1,0,0,1,1
Movie5,0,1,0,1,0,1,0,0
Movie6,1,0,0,1,1,0,0,1
Movie7,0,0,1,1,0,0,0,0
Movie8,0,1,0,0,1,1,0,0
Movie9,1,0,0,1,1,0,1,1
Movie10,0,1,1,0,1,0,0,1


In [2]:
#Get the our Rating and our seen and unseen movies
#

#get number of movies
movies = len(rating)
#get our ratings
ourRating = rating.head(movies).ix[:,0:1]

#print our ratings
print(ourRating)


#create 2 empty lists
notSeenMovies = []
seenMovies = []

#fill both list with seen or unseen movies
for i in range(0,movies):
    if (ourRating.iloc[i]["We"] == 0):
         notSeenMovies.append(i)
    else:
        seenMovies.append(i)
        
#print those movies       
print ("seen movies:")
print (seenMovies)
              
print ("not yet seen movies:")
print (notSeenMovies)


         We
Movie1    0
Movie2    0
Movie3    1
Movie4    1
Movie5    0
Movie6    1
Movie7    0
Movie8    0
Movie9    1
Movie10   0
seen movies:
[2, 3, 5, 8]
not yet seen movies:
[0, 1, 4, 6, 7, 9]


In [3]:
#calculating similaries from each item
dist_out = 1-pairwise_distances(rating, metric="cosine")

#print the similarities
dist_out

array([[ 1.        ,  1.        ,  0.77459667,  0.51639778,  0.66666667,
         0.        ,  0.        ,  0.66666667,  0.25819889,  0.28867513],
       [ 1.        ,  1.        ,  0.77459667,  0.51639778,  0.66666667,
         0.        ,  0.        ,  0.66666667,  0.25819889,  0.28867513],
       [ 0.77459667,  0.77459667,  1.        ,  0.6       ,  0.51639778,
         0.2236068 ,  0.31622777,  0.51639778,  0.4       ,  0.4472136 ],
       [ 0.51639778,  0.51639778,  0.6       ,  1.        ,  0.51639778,
         0.67082039,  0.31622777,  0.25819889,  0.8       ,  0.4472136 ],
       [ 0.66666667,  0.66666667,  0.51639778,  0.51639778,  1.        ,
         0.28867513,  0.40824829,  0.66666667,  0.25819889,  0.28867513],
       [ 0.        ,  0.        ,  0.2236068 ,  0.67082039,  0.28867513,
         1.        ,  0.35355339,  0.28867513,  0.89442719,  0.5       ],
       [ 0.        ,  0.        ,  0.31622777,  0.31622777,  0.40824829,
         0.35355339,  1.        ,  0.        

In [4]:
#output as sparse for better visuality
A_sparse = sparse.csr_matrix(rating)

#calculating similarities for sparse output
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)

#print the sparse similarities
print (similarities_sparse)



  (0, 8)	0.258198889747
  (0, 9)	0.288675134595
  (0, 7)	0.666666666667
  (0, 4)	0.666666666667
  (0, 3)	0.516397779494
  (0, 2)	0.774596669241
  (0, 1)	1.0
  (0, 0)	1.0
  (1, 8)	0.258198889747
  (1, 9)	0.288675134595
  (1, 7)	0.666666666667
  (1, 4)	0.666666666667
  (1, 3)	0.516397779494
  (1, 2)	0.774596669241
  (1, 1)	1.0
  (1, 0)	1.0
  (2, 6)	0.316227766017
  (2, 9)	0.4472135955
  (2, 7)	0.516397779494
  (2, 4)	0.516397779494
  (2, 1)	0.774596669241
  (2, 0)	0.774596669241
  (2, 8)	0.4
  (2, 5)	0.22360679775
  (2, 3)	0.6
  :	:
  (7, 4)	0.666666666667
  (7, 3)	0.258198889747
  (7, 2)	0.516397779494
  (7, 1)	0.666666666667
  (7, 0)	0.666666666667
  (8, 1)	0.258198889747
  (8, 0)	0.258198889747
  (8, 9)	0.4472135955
  (8, 7)	0.258198889747
  (8, 6)	0.316227766017
  (8, 4)	0.258198889747
  (8, 8)	1.0
  (8, 5)	0.894427191
  (8, 3)	0.8
  (8, 2)	0.4
  (9, 8)	0.4472135955
  (9, 5)	0.5
  (9, 6)	0.353553390593
  (9, 9)	1.0
  (9, 7)	0.57735026919
  (9, 4)	0.288675134595
  (9, 3)	0.4472135955


In [5]:
#Get the neighbors and sort them descending
#

#creating array for the neighbors
w = 3
h = len(seenMovies)*len(notSeenMovies)
a = [[0 for x in range(w)] for y in range(h)] 

#counter which helps filling the neighbor array
counter = -1

#filling array with zeros
for i in range(0,len(seenMovies)):
    for j in range(0,3):
        a[i][j] = 0

#filling array with seen, unseen and similarity
for i in range(0,len(seenMovies)):
    for j in range(0,len(notSeenMovies)):
        counter+=1
        print("Sim",seenMovies[i],notSeenMovies[j],dist_out[seenMovies[i]][notSeenMovies[j]])
        a[counter][0] = seenMovies[i]
        a[counter][1] = notSeenMovies[j]
        a[counter][2] = dist_out[seenMovies[i]][notSeenMovies[j]]
   
#sort Array by similarity
a = sorted(a,key=lambda x: x[2], reverse=True)

#print results
print("\nMovieX, MovieY, Sim")
for i in range(0,len(a)):
    print("\n")
    for j in range(0,len(a[i])):
        print(a[i][j], end="       ")
        

Sim 2 0 0.774596669241
Sim 2 1 0.774596669241
Sim 2 4 0.516397779494
Sim 2 6 0.316227766017
Sim 2 7 0.516397779494
Sim 2 9 0.4472135955
Sim 3 0 0.516397779494
Sim 3 1 0.516397779494
Sim 3 4 0.516397779494
Sim 3 6 0.316227766017
Sim 3 7 0.258198889747
Sim 3 9 0.4472135955
Sim 5 0 0.0
Sim 5 1 0.0
Sim 5 4 0.288675134595
Sim 5 6 0.353553390593
Sim 5 7 0.288675134595
Sim 5 9 0.5
Sim 8 0 0.258198889747
Sim 8 1 0.258198889747
Sim 8 4 0.258198889747
Sim 8 6 0.316227766017
Sim 8 7 0.258198889747
Sim 8 9 0.4472135955

MovieX, MovieY, Sim


2       0       0.774596669241       

2       1       0.774596669241       

2       4       0.516397779494       

2       7       0.516397779494       

3       0       0.516397779494       

3       1       0.516397779494       

3       4       0.516397779494       

5       9       0.5       

2       9       0.4472135955       

3       9       0.4472135955       

8       9       0.4472135955       

5       6       0.353553390593       

2       6    

In [6]:
#removing some dublicates from the neighbor array and printing recommendations
#

#needed variables and 2 new arrays
j = 2
k = 30
counter2 = 0
seen = -1
b = [[0 for x in range(j)] for y in range(k)] 
c = []


#get highest neighbors and put it in 2 arrays
for i in range(0,len(a)):
     if (seen != a[i][1]):
        b[counter2][0] = a[i][1]
        b[counter2][1] = a[i][2]
        c.append(a[i][1])
        seen = a[i][1]
        counter2+=1

#time for recommendations 
print("Recommended 3-NN")
for i in range(0,3):
    print("Movie",(b[i][0])+1, "sim: ", b[i][1])
   
print("\nRecommended items with sim > 0.7")  
for i in range(0,3):
    if (b[i][1] > 0.7):
        print("Movie",(b[i][0])+1, "sim: ", b[i][1])    
 

print("\nall possible recommendations")
c = list(set(c))
for i in range(0,len(c)):
    print("Movie",c[i]+1)
    
    


Recommended 3-NN
Movie 1 sim:  0.774596669241
Movie 2 sim:  0.774596669241
Movie 5 sim:  0.516397779494

Recommended items with sim > 0.7
Movie 1 sim:  0.774596669241
Movie 2 sim:  0.774596669241

all possible recommendations
Movie 1
Movie 2
Movie 5
Movie 7
Movie 8
Movie 10
